# Minimal Reproduction: Hodler & Raschky (2014)
# *Regional Favoritism*

**Main specification:** Do leaders channel economic resources to their birth regions?

$$\ln(\text{light}_{i,t}) = \beta \cdot \text{BirthRegionLeader}_{i,t} + \alpha_i + \gamma_{c,t} + \varepsilon_{i,t}$$

- $\alpha_i$: region (ADM2) fixed effects  
- $\gamma_{c,t}$: country $\times$ year fixed effects  
- $\text{BirthRegionLeader}_{i,t} = 1$ if region $i$ is the birth region of the national leader in year $t$

**Data:**
- Political leaders: PLAD (Bomprezzi et al., 2025)
- Nightlights: Harmonized DMSP-OLS / VIIRS (Li et al., 2020)
- Admin boundaries: GADM v4.1 (ADM2 level, ~39,000 districts)

In [38]:
import os
import warnings
import zipfile
import urllib.request
from pathlib import Path

import numpy as np
import pandas as pd
import geopandas as gpd
import rasterio
from rasterstats import zonal_stats
from linearmodels.panel import PanelOLS

warnings.filterwarnings("ignore")

ROOT = Path("..")  # project root
DATA = ROOT / "data"
NTL_DIR = DATA / "nightlights"
PLAD_PATH = DATA / "political leaders" / "PLAD_April_2024.dta"

## 1. Download GADM ADM2 boundaries

In [39]:
GADM_DIR = DATA / "gadm"
GADM_GPKG = GADM_DIR / "gadm_410.gpkg"
ADM2_CACHE = GADM_DIR / "gadm41_adm2.gpkg"

# --- Download GADM if needed ---
if not GADM_GPKG.exists() and not ADM2_CACHE.exists():
    GADM_DIR.mkdir(parents=True, exist_ok=True)
    url = "https://geodata.ucdavis.edu/gadm/gadm4.1/gadm_410-gpkg.zip"
    zip_path = GADM_DIR / "gadm_410-gpkg.zip"
    print("Downloading GADM (~1.4 GB) — this may take a few minutes...")
    urllib.request.urlretrieve(url, zip_path)
    print("Extracting...")
    with zipfile.ZipFile(zip_path, "r") as zf:
        zf.extractall(GADM_DIR)
    zip_path.unlink()
    print("Done.")

# --- Dissolve to ADM2 level and cache ---
if ADM2_CACHE.exists():
    print(f"Loading cached ADM2 boundaries from {ADM2_CACHE}")
    adm2 = gpd.read_file(ADM2_CACHE)
else:
    print("Reading GADM and dissolving to ADM2 (may take several minutes)...")
    raw = gpd.read_file(GADM_GPKG)

    # For countries that lack ADM2 subdivision, promote their ADM1 code to GID_2
    # (mirrors Bora 2025: "retain ADM1 regions for countries with no ADM2")
    has_gid2 = raw["GID_2"].notna() & (raw["GID_2"].str.strip() != "")
    # Replace GID_2 in-place so dissolve key has no duplicate column after the operation
    raw["GID_2"] = raw["GID_2"].where(has_gid2, raw["GID_1"])

    adm2 = raw.dissolve(by="GID_2", as_index=False)
    keep_cols = [c for c in ["GID_0", "GID_1", "GID_2", "NAME_0", "NAME_1", "NAME_2", "geometry"]
                 if c in adm2.columns]
    adm2 = adm2[keep_cols]
    adm2.to_file(ADM2_CACHE, driver="GPKG")
    print(f"Cached ADM2 boundaries to {ADM2_CACHE}")

print(f"Loaded {len(adm2)} ADM2 regions")
adm2.head(2)

Reading GADM and dissolving to ADM2 (may take several minutes)...
Cached ADM2 boundaries to ../data/gadm/gadm41_adm2.gpkg
Loaded 47986 ADM2 regions


,GID_0,GID_1,GID_2,NAME_0,NAME_1,NAME_2,geometry
0,ATA,,,Antarctica,,,"MULTIPOLYGON (((-169.00626 -83.61875, -169.004..."
1,UKR,?,?,Ukraine,?,?,"POLYGON ((30.59167 50.41236, 30.60611 50.41604..."


## 2. Compute mean nightlights per ADM2-year (zonal statistics)

We compute the mean nightlight intensity within each ADM2 polygon for each year of the harmonized DMSP rasters (1992–2013, matching the original paper's DMSP era).

Using ADM2 (~39,000 districts) instead of ADM1 (~3,600 provinces) gives ~11× more regions, providing many more control units per treated birth district and substantially more statistical power.

In [40]:
PANEL_CACHE = DATA / "nightlights_adm2_panel.parquet"
YEARS = range(1992, 2014)  # 1992-2013 inclusive (DMSP era)

def raster_path(year):
    """Return the path to the harmonized nightlights raster for a given year."""
    if year <= 2013:
        return NTL_DIR / f"Harmonized_DN_NTL_{year}_calDMSP.tif"
    else:
        return NTL_DIR / f"Harmonized_DN_NTL_{year}_simVIIRS.tif"

if PANEL_CACHE.exists():
    print(f"Loading cached panel from {PANEL_CACHE}")
    ntl_panel = pd.read_parquet(PANEL_CACHE)
else:
    frames = []
    for year in YEARS:
        rpath = raster_path(year)
        if not rpath.exists():
            print(f"  Skipping {year} — raster not found")
            continue
        print(f"  Processing {year}...", end=" ", flush=True)
        stats = zonal_stats(
            adm2.geometry,
            str(rpath),
            stats=["mean"],
            nodata=0,
        )
        means = [s["mean"] for s in stats]
        df_year = pd.DataFrame({
            "GID_2": adm2["GID_2"],
            "GID_1": adm2["GID_1"],
            "GID_0": adm2["GID_0"],
            "year": year,
            "ntl_mean": means,
        })
        frames.append(df_year)
        print(f"done (non-null: {df_year['ntl_mean'].notna().sum()})")

    ntl_panel = pd.concat(frames, ignore_index=True)
    ntl_panel.to_parquet(PANEL_CACHE)
    print(f"\nSaved panel to {PANEL_CACHE}")

print(f"Panel shape: {ntl_panel.shape}")
ntl_panel.head()

  Processing 1992... done (non-null: 38457)
  Processing 1993... done (non-null: 41595)
  Processing 1994... done (non-null: 41881)
  Processing 1995... done (non-null: 42383)
  Processing 1996... done (non-null: 42770)
  Processing 1997... done (non-null: 42762)
  Processing 1998... done (non-null: 43119)
  Processing 1999... done (non-null: 43304)
  Processing 2000... done (non-null: 43501)
  Processing 2001... done (non-null: 43442)
  Processing 2002... done (non-null: 42531)
  Processing 2003... done (non-null: 43383)
  Processing 2004... done (non-null: 44177)
  Processing 2005... done (non-null: 43970)
  Processing 2006... done (non-null: 43589)
  Processing 2007... done (non-null: 44227)
  Processing 2008... done (non-null: 44016)
  Processing 2009... done (non-null: 43237)
  Processing 2010... done (non-null: 45052)
  Processing 2011... done (non-null: 45030)
  Processing 2012... done (non-null: 44764)
  Processing 2013... done (non-null: 44769)

Saved panel to ../data/nightlig

,GID_2,GID_1,GID_0,year,ntl_mean
0,,,ATA,1992,11.641598
1,?,?,UKR,1992,27.016260
2,AFG.1.10_1,AFG.1_1,AFG,1992,NaN
3,AFG.1.11_1,AFG.1_1,AFG,1992,NaN
4,AFG.1.12_1,AFG.1_1,AFG,1992,NaN


## 3. Build treatment variable from PLAD

In [41]:
plad = pd.read_stata(PLAD_PATH)

# Keep domestic leaders with valid birth region identifiers
plad = plad[plad["foreign_leader"] == "0"].copy()

# Detect which GID level is available for birth region coding
gid_cols = [c for c in plad.columns if c.lower().startswith("gid_")]
print(f"GID columns in PLAD: {gid_cols}")

# Prefer gid_2 (ADM2 birth district, matching H&R/Bora); fall back to gid_1 (ADM1 province)
if "gid_2" in plad.columns:
    n_valid_gid2 = (plad["gid_2"].str.strip() != ".").sum()
    print(f"gid_2 non-missing: {n_valid_gid2} / {len(plad)}")
    BIRTH_GID = "gid_2"
    print("Using ADM2 birth regions (gid_2) — matches H&R / Bora 2025")
else:
    BIRTH_GID = "gid_1"
    print("WARNING: gid_2 not in PLAD — using gid_1 (ADM1). Treatment will be assigned to "
          "all ADM2 districts within the birth province.")

plad = plad[plad[BIRTH_GID].str.strip() != "."].copy()
plad["startyear"] = plad["startyear"].astype(int)
plad["endyear"] = plad["endyear"].astype(int)

# Restrict to leaders with an Archigos ID — matches H&R's "effective executive" definition
plad = plad[plad["archigos_id"].str.strip() != "."].copy()

# Sort by country then start year (required for overlap fix below)
plad = plad.sort_values(["gid_0", "startyear"]).reset_index(drop=True)

print(f"Leaders after filtering: {len(plad)}")
print(f"Unique countries: {plad['country'].nunique()}")
print(f"Year range: {plad['startyear'].min()}–{plad['endyear'].max()}")
plad[["leader", "country", BIRTH_GID, "startyear", "endyear"]].head(10)

GID columns in PLAD: ['gid_0', 'gid_1', 'gid_2']
gid_2 non-missing: 1025 / 1253
Using ADM2 birth regions (gid_2) — matches H&R / Bora 2025
Leaders after filtering: 820
Unique countries: 151
Year range: 1948–2023


,leader,country,gid_2,startyear,endyear
0,Najibullah,Afghanistan,AFG.26.4_1,1986,1992
1,Mojadidi,Afghanistan,AFG.14.6_1,1992,1992
2,Mullah Omar,Afghanistan,AFG.15.6_1,1996,2001
3,Hamid Karzai,Afghanistan,AFG.15.5_1,2001,2014
4,Ashraf Ghani Ahmadzai,Afghanistan,AFG.14.8_1,2014,2021
5,Dos Santos,Angola,AGO.11.8_1,1979,2017
6,Alia,Albania,ALB.10.3_1,1985,1992
7,Berisha,Albania,ALB.8.3_1,1992,1997
8,Fatos Nano,Albania,ALB.11.2_1,1997,1998
9,Majko,Albania,ALB.11.2_1,1998,1999


In [42]:
# --- Fix 1: Resolve transition-year overlap ---
# PLAD codes outgoing leader's endyear == incoming leader's startyear,
# so naively both are treated in the same year. We clip each leader's
# endyear to (next leader's startyear - 1) within the same country.
plad_fixed = plad.copy().reset_index(drop=True)

for gid_0, group in plad_fixed.groupby("gid_0"):
    idxs = group.index.tolist()
    for i in range(len(idxs) - 1):
        curr, nxt = idxs[i], idxs[i + 1]
        if plad_fixed.loc[curr, "endyear"] >= plad_fixed.loc[nxt, "startyear"]:
            plad_fixed.loc[curr, "endyear"] = plad_fixed.loc[nxt, "startyear"] - 1

n_fixed = (plad_fixed["endyear"] != plad["endyear"]).sum()
print(f"Overlapping spells clipped: {n_fixed}")

# Expand spells into birth-region × year rows
rows = []
for idx, row in plad_fixed.iterrows():
    for y in range(max(int(row["startyear"]), 1992), min(int(row["endyear"]), 2013) + 1):
        rows.append({"GID_2": row[BIRTH_GID], "GID_0": row["gid_0"], "year": y, "spell_id": idx})

leader_years = pd.DataFrame(rows)

# If PLAD only has ADM1 codes, expand each birth ADM1 to all constituent ADM2 districts
if BIRTH_GID == "gid_1":
    adm1_to_adm2 = ntl_panel[["GID_2", "GID_1"]].drop_duplicates()
    leader_years = (
        leader_years
        .rename(columns={"GID_2": "GID_1"})
        .merge(adm1_to_adm2, on="GID_1", how="inner")
        [["GID_2", "GID_0", "year", "spell_id"]]
    )
    print(f"Expanded ADM1 birth regions → {len(leader_years)} ADM2-year rows")

# Sanity check: at most 1 treated leader per country-year (only meaningful for ADM2 birth codes)
multi = leader_years.groupby(["GID_0", "year"])["GID_2"].nunique()
n_multi = (multi > 1).sum()
print(f"Country-years with >1 treated region: {n_multi}"
      + ("  (should be 0)" if BIRTH_GID == "gid_2" else "  (expected for ADM1 expansion)"))

leader_years = leader_years.drop_duplicates(subset=["GID_2", "year"])
leader_years["birth_region_leader"] = 1
print(f"Birth-region × year observations: {len(leader_years)}")

# --- Build country-year → leader-spell mapping for SE clustering ---
# All regions in a country share the same cluster ID for a given leader's tenure.
spell_map = leader_years[["GID_0", "year", "spell_id"]].drop_duplicates(subset=["GID_0", "year"])
print(f"Unique leader spells active in panel: {spell_map['spell_id'].nunique()}")

Overlapping spells clipped: 590
Country-years with >1 treated region: 0  (should be 0)
Birth-region × year observations: 3002
Unique leader spells active in panel: 631


In [43]:
# --- Fix 2: GID_2 match rate diagnostic ---
# Check how many PLAD birth region codes actually match GADM GID_2 codes in the panel.
plad_gids  = set(plad_fixed[BIRTH_GID].unique())
panel_gids = set(ntl_panel["GID_2"].unique())

matched   = plad_gids & panel_gids
unmatched = plad_gids - panel_gids

level_label = "GID_2" if BIRTH_GID == "gid_2" else "GID_1→GID_2"
print(f"Unique {level_label} codes in PLAD:              {len(plad_gids)}")
print(f"Unique GID_2 codes in nightlights panel:         {len(panel_gids)}")
print(f"Matched (appear in both):                        {len(matched)}  ({len(matched)/len(plad_gids):.1%})")
print(f"Unmatched (PLAD birth regions with no NTL data): {len(unmatched)}")

if unmatched:
    unmatched_leaders = plad_fixed[plad_fixed[BIRTH_GID].isin(unmatched)][
        ["leader", "country", BIRTH_GID]
    ]
    print(f"\nLeaders with unmatched birth regions: {len(unmatched_leaders)}")
    print(unmatched_leaders.to_string())

Unique GID_2 codes in PLAD:              579
Unique GID_2 codes in nightlights panel:         47986
Matched (appear in both):                        538  (92.9%)
Unmatched (PLAD birth regions with no NTL data): 41

Leaders with unmatched birth regions: 58
                      leader      country         gid_2
31                 Vranitzky      Austria     AUT.9.1_1
32                     Klima      Austria    AUT.3.14_1
33                  Schussel      Austria     AUT.9.1_1
34                Gusenbauer      Austria    AUT.3.17_1
35                   Faymann      Austria     AUT.9.1_1
66                     Ahmed   Bangladesh    BGD.3.13_1
108               Paz Zamora      Bolivia     BOL.2.6_1
109            Banzer Suarez      Bolivia     BOL.8.1_1
110    Jorge Quiroga Ramirez      Bolivia     BOL.2.6_1
111       Enrique RodrÃ­guez      Bolivia     BOL.2.6_1
112             Juan Morales      Bolivia     BOL.5.9_1
113                   Sarney       Brazil  BRA.10.144_1
114             

In [44]:
# Merge treatment onto panel
panel = ntl_panel.merge(leader_years[["GID_2", "year", "birth_region_leader"]],
                        on=["GID_2", "year"], how="left")
panel["birth_region_leader"] = panel["birth_region_leader"].fillna(0).astype(int)

# Merge leader-period spell ID for clustering (all regions in a country-year
# share the cluster of whichever leader was in power that year)
panel = panel.merge(spell_map[["GID_0", "year", "spell_id"]], on=["GID_0", "year"], how="left")
# Fallback: country-year string for obs with no leader record (acts as singleton clusters)
panel["spell_id"] = panel["spell_id"].fillna(
    panel["GID_0"] + "_" + panel["year"].astype(str) + "_noleader"
).astype(str)

# Add lag 1 and lag 2 of treatment (computed before dropping NaN rows to
# avoid incorrectly skipping years with missing nightlights)
panel = panel.sort_values(["GID_2", "year"])
panel["brl_lag1"] = panel.groupby("GID_2")["birth_region_leader"].shift(1).fillna(0).astype(int)
panel["brl_lag2"] = panel.groupby("GID_2")["birth_region_leader"].shift(2).fillna(0).astype(int)

# Construct log nightlights and country-year FE identifier
panel["ln_ntl"] = np.log(panel["ntl_mean"] + 0.01)
panel["country_year"] = panel["GID_0"] + "_" + panel["year"].astype(str)

# Drop rows with missing nightlights
panel = panel.dropna(subset=["ntl_mean"])

print(f"Final panel: {panel.shape[0]} obs, {panel['GID_2'].nunique()} regions, "
      f"{panel['year'].nunique()} years")
print(f"Treated obs (birth_region_leader=1): {panel['birth_region_leader'].sum()}")
print(f"Treated obs (lag 1):                 {panel['brl_lag1'].sum()}")
print(f"Treated obs (lag 2):                 {panel['brl_lag2'].sum()}")
print(f"Unique leader-period clusters:       {panel['spell_id'].nunique()}")
panel.head()

Final panel: 951959 obs, 46568 regions, 22 years
Treated obs (birth_region_leader=1): 2522
Treated obs (lag 1):                 2406
Treated obs (lag 2):                 2297
Unique leader-period clusters:       2745


,GID_2,GID_1,GID_0,year,ntl_mean,birth_region_leader,spell_id,brl_lag1,brl_lag2,ln_ntl,country_year
0,,,ATA,1992,11.641598,0,ATA_1992_noleader,0,0,2.455443,ATA_1992
47986,,,ATA,1993,9.313452,0,ATA_1993_noleader,0,0,2.232533,ATA_1993
95972,,,ATA,1994,8.455028,0,ATA_1994_noleader,0,0,2.135943,ATA_1994
143958,,,ATA,1995,10.064439,0,ATA_1995_noleader,0,0,2.310001,ATA_1995
191944,,,ATA,1996,9.032386,0,ATA_1996_noleader,0,0,2.201923,ATA_1996


## 4. Main regression

$$\ln(\text{light}_{i,t}) = \beta \cdot \text{BirthRegionLeader}_{i,t} + \alpha_i + \gamma_{c,t} + \varepsilon_{i,t}$$

Standard errors clustered at the country level, following Hodler & Raschky (2014).

In [45]:
# Main specification: region FE + country×year FE
# Clustered at leader-period level (Bora 2025 / H&R convention)
if not isinstance(panel.index, pd.MultiIndex):
    panel = panel.set_index(["GID_2", "year"])

specs = [
    ("Lag 0", "birth_region_leader"),
    ("Lag 1", "brl_lag1"),
    ("Lag 2", "brl_lag2"),
]

rows = []
for label, var in specs:
    model = PanelOLS.from_formula(
        f"ln_ntl ~ {var} + EntityEffects",
        data=panel,
        other_effects=panel["country_year"],
        drop_absorbed=True,
    )
    res = model.fit(cov_type="clustered", clusters=panel["spell_id"])
    coef  = res.params[var]
    se    = res.std_errors[var]
    pval  = res.pvalues[var]
    stars = "***" if pval < 0.01 else "**" if pval < 0.05 else "*" if pval < 0.1 else ""
    rows.append({
        "Spec":        label,
        "Coeff":       f"{coef:.4f}{stars}",
        "Std. Err.":   f"({se:.4f})",
        "p-value":     f"{pval:.3f}",
        "N":           f"{res.nobs:,}",
    })

import pandas as pd
print("Dep. var: log(nightlights + 0.01)")
print("FE: Region (ADM2) + Country×Year  |  Clustering: Leader-period\n")
print(pd.DataFrame(rows).set_index("Spec").to_string())
print("\nSignificance: *** p<0.01, ** p<0.05, * p<0.1")

Dep. var: log(nightlights + 0.01)
FE: Region (ADM2) + Country×Year  |  Clustering: Leader-period

           Coeff Std. Err. p-value        N
Spec                                       
Lag 0   0.0128**  (0.0059)   0.030  951,959
Lag 1  0.0171***  (0.0056)   0.002  951,959
Lag 2  0.0143***  (0.0055)   0.010  951,959

Significance: *** p<0.01, ** p<0.05, * p<0.1


## 5. Democracy interaction

Hodler & Raschky's key finding is that regional favoritism is concentrated in **autocracies**. We interact `birth_region_leader` with a democracy index to test this.

**Data:** V-Dem Electoral Democracy Index (`v2x_polyarchy`, 0–1).  
Download the **Country-Year Core** CSV from [v-dem.net/data](https://v-dem.net/data/the-v-dem-dataset/) (free, requires quick registration) and place the CSV file in `data/vdem/`.

In [46]:
VDEM_DIR = DATA / "vdem"
vdem_csv = list(VDEM_DIR.glob("*.csv")) if VDEM_DIR.exists() else []
assert vdem_csv, (
    "V-Dem CSV not found. Download the Country-Year Core CSV from "
    "https://v-dem.net/data/the-v-dem-dataset/ and place it in data/vdem/"
)

vdem = pd.read_csv(vdem_csv[0], low_memory=False)
# Keep only what we need: country code, year, polyarchy index
vdem = vdem[["country_text_id", "year", "v2x_polyarchy"]].rename(
    columns={"country_text_id": "GID_0", "v2x_polyarchy": "democracy"}
)
vdem = vdem.dropna(subset=["democracy"])
vdem = vdem[(vdem["year"] >= 1992) & (vdem["year"] <= 2013)]

print(f"V-Dem: {len(vdem)} country-year obs, {vdem['GID_0'].nunique()} countries")
vdem.head()

V-Dem: 3890 country-year obs, 179 countries


,GID_0,year,democracy
203,MEX,1992,0.437
204,MEX,1993,0.447
205,MEX,1994,0.470
206,MEX,1995,0.480
207,MEX,1996,0.508


In [47]:
# Merge democracy scores onto panel and build interactions
panel_dem = panel.reset_index().merge(vdem, on=["GID_0", "year"], how="left")
panel_dem = panel_dem.dropna(subset=["democracy"])

panel_dem["birth_x_democracy"]     = panel_dem["birth_region_leader"] * panel_dem["democracy"]
panel_dem["brl_lag1_x_democracy"]  = panel_dem["brl_lag1"] * panel_dem["democracy"]
panel_dem["brl_lag2_x_democracy"]  = panel_dem["brl_lag2"] * panel_dem["democracy"]

print(f"Panel with democracy: {len(panel_dem)} obs")
panel_dem = panel_dem.set_index(["GID_2", "year"])

# Interaction specification:
# ln(light) = β1·BirthRegion + β2·BirthRegion×Democracy + α_i + γ_ct + ε
# β1 > 0: favoritism in full autocracies (democracy = 0)
# β2 < 0: favoritism attenuated in democracies

specs_dem = [
    ("Lag 0", "birth_region_leader", "birth_x_democracy"),
    ("Lag 1", "brl_lag1",            "brl_lag1_x_democracy"),
    ("Lag 2", "brl_lag2",            "brl_lag2_x_democracy"),
]

rows_dem = []
for label, var, inter in specs_dem:
    model_dem = PanelOLS.from_formula(
        f"ln_ntl ~ {var} + {inter} + EntityEffects",
        data=panel_dem,
        other_effects=panel_dem["country_year"],
        drop_absorbed=True,
    )
    res = model_dem.fit(cov_type="clustered", clusters=panel_dem["spell_id"])

    for name, key in [("BirthRegion", var), ("BirthRegion×Democracy", inter)]:
        coef  = res.params[key]
        se    = res.std_errors[key]
        pval  = res.pvalues[key]
        stars = "***" if pval < 0.01 else "**" if pval < 0.05 else "*" if pval < 0.1 else ""
        rows_dem.append({
            "Spec": label, "Variable": name,
            "Coeff": f"{coef:.4f}{stars}", "Std. Err.": f"({se:.4f})", "p-value": f"{pval:.3f}",
        })

print("\nDep. var: log(nightlights + 0.01)")
print("FE: Region (ADM2) + Country×Year  |  Clustering: Leader-period\n")
print(pd.DataFrame(rows_dem).set_index(["Spec", "Variable"]).to_string())
print("\nSignificance: *** p<0.01, ** p<0.05, * p<0.1")

Panel with democracy: 934637 obs

Dep. var: log(nightlights + 0.01)
FE: Region (ADM2) + Country×Year  |  Clustering: Leader-period

                                Coeff Std. Err. p-value
Spec  Variable                                         
Lag 0 BirthRegion              0.0150  (0.0152)   0.322
      BirthRegion×Democracy   -0.0043  (0.0210)   0.839
Lag 1 BirthRegion            0.0311**  (0.0144)   0.031
      BirthRegion×Democracy   -0.0238  (0.0194)   0.220
Lag 2 BirthRegion            0.0336**  (0.0150)   0.025
      BirthRegion×Democracy   -0.0325  (0.0201)   0.106

Significance: *** p<0.01, ** p<0.05, * p<0.1


In [ ]:
# Load NO₂ panel from downloaded CSV (run after GEE task completes)

if NO2_PANEL_CACHE.exists():
    print(f"Loading cached NO₂ panel from {NO2_PANEL_CACHE}")
    no2_panel = pd.read_parquet(NO2_PANEL_CACHE)
else:
    assert NO2_CSV.exists(), (
        f"NO₂ CSV not found at {NO2_CSV}.\n"
        f"Complete the GEE task above and download 'no2_adm2_annual.csv'\n"
        f"from Google Drive (folder: {DRIVE_FOLDER}) into {DATA}/"
    )
    no2_panel = pd.read_csv(NO2_CSV)

    # Rename GEE mean column; drop rows where GEE returned no data (ocean, ice)
    no2_panel = no2_panel.rename(columns={"no2": "no2_mean"})
    no2_panel = no2_panel.dropna(subset=["no2_mean"])
    no2_panel = no2_panel[no2_panel["no2_mean"] >= 0]   # remove fill values

    # Convert mol/m² → µmol/m² for interpretability
    no2_panel["no2_mean"] = no2_panel["no2_mean"] * 1e6

    no2_panel.to_parquet(NO2_PANEL_CACHE)
    print(f"Saved NO₂ panel to {NO2_PANEL_CACHE}")

print(f"NO₂ panel: {len(no2_panel):,} obs | "
      f"{no2_panel['GID_2'].nunique():,} regions | "
      f"years: {sorted(no2_panel['year'].unique())}")
no2_panel.head()